FATIMA BABAR (22I-1565)

OSAED JAFAR (22I-1776)

CY-A

AI FINAL PROJECT

preprocessing

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import os
import json

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_summary(text):
    """Clean and preprocess movie summary text"""
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text

def parse_genres(genre_str):
    """Parse genre information from the metadata format"""
    if not isinstance(genre_str, str) or genre_str == "":
        return []
    
    try:
        if genre_str.startswith('{') and not genre_str.startswith('{"'):
            genre_str = genre_str.replace("'", '"')
            genre_str = re.sub(r'([{,])\s*([^"{\s][^:]*?):', r'\1"\2":', genre_str)
        
        try:
            data = json.loads(genre_str)
            if isinstance(data, dict):
                return list(data.values())
        except json.JSONDecodeError:
            genres = []
            matches = re.findall(r'"[^"]+"\s*:\s*"([^"]+)"', genre_str)
            for match in matches:
                genres.append(match)
            return genres
    except Exception as e:
        print(f"Error parsing genres: {e}")
        return []

def process_movie_dataset(plot_summaries_path, metadata_path, output_path):
    print("Loading datasets...")
    summaries_df = pd.read_csv(plot_summaries_path, sep='\t', header=None, names=['WikipediaID', 'Summary'])
    metadata_df = pd.read_csv(metadata_path, sep='\t', header=None, names=[
        'WikipediaID', 'FreebaseID', 'Title', 'ReleaseDate', 
        'Revenue', 'Runtime', 'Languages', 'Countries', 'Genres'])
    
    print(f"Loaded {len(summaries_df)} summaries and {len(metadata_df)} metadata entries")

    print("\nSample of genre data format:")
    for sample in metadata_df['Genres'].head(3).tolist():
        print(f"Raw genre data: {sample}")
        parsed = parse_genres(sample)
        print(f"Parsed genres: {parsed}")
        print("-" * 50)

    print("Cleaning summaries...")
    summaries_df['CleanedSummary'] = summaries_df['Summary'].apply(preprocess_summary)

    print("Extracting genres...")
    metadata_df['GenreList'] = metadata_df['Genres'].apply(parse_genres)
    print(f"Entries with non-empty genre lists: {(metadata_df['GenreList'].apply(len) > 0).sum()}")

    summaries_df['WikipediaID'] = summaries_df['WikipediaID'].astype(str)
    metadata_df['WikipediaID'] = metadata_df['WikipediaID'].astype(str)

    print("Merging datasets...")
    merged_df = pd.merge(summaries_df, metadata_df[['WikipediaID', 'Title', 'GenreList']], on='WikipediaID', how='inner')
    print(f"Merged dataset contains {len(merged_df)} entries")

    empty_genres = (merged_df['GenreList'].apply(len) == 0).sum()
    short_summaries = (merged_df['CleanedSummary'].str.len() <= 10).sum()
    print(f"Entries with empty genres: {empty_genres}")
    print(f"Entries with short summaries: {short_summaries}")

    merged_df = merged_df[
        (merged_df['GenreList'].apply(len) > 0) & 
        (merged_df['CleanedSummary'].str.len() > 10)
    ]
    print(f"After filtering, dataset contains {len(merged_df)} entries")

    merged_df['GenreCount'] = merged_df['GenreList'].apply(len)
    genre_count_dist = merged_df['GenreCount'].value_counts().sort_index()
    print("\nDistribution of genre counts:")
    print(genre_count_dist)

    # Ensure output folder exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    merged_df['GenreList'] = merged_df['GenreList'].apply(json.dumps)
    merged_df.to_csv(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path}")

    # Convert back to list for further analysis
    merged_df['GenreList'] = merged_df['GenreList'].apply(json.loads)
    return merged_df

def main():
    plot_summaries_path = os.path.join('dataset', 'plot_summaries.txt')
    metadata_path = os.path.join('dataset', 'movie.metadata.tsv')
    output_path = os.path.join('output', 'cleaned_movie_data.csv')

    if not os.path.exists(plot_summaries_path):
        print(f"Error: {plot_summaries_path} not found.")
        return
    if not os.path.exists(metadata_path):
        print(f"Error: {metadata_path} not found.")
        return

    processed_df = process_movie_dataset(plot_summaries_path, metadata_path, output_path)

    print("Data preprocessing complete!")
    print(f"Total movies processed: {len(processed_df)}")

    print("\nSample of processed data:")
    if len(processed_df) > 0:
        sample = processed_df.sample(min(5, len(processed_df)))
        for _, row in sample.iterrows():
            print(f"Title: {row['Title']}")
            print(f"Genres: {row['GenreList']}")
            print(f"Summary: {row['CleanedSummary'][:100]}...")
            print("-" * 50)
    else:
        print("No data to sample - dataset is empty.")

    if len(processed_df) > 0:
        all_genres = []
        for genres in processed_df['GenreList']:
            all_genres.extend(genres)
        genre_counts = pd.Series(all_genres).value_counts()
        print("\nTop genres in the dataset:")
        print(genre_counts.head(10))

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to C:\Users\Dell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dell/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading datasets...
Loaded 42303 summaries and 81741 metadata entries

Sample of genre data format:
Raw genre data: {"/m/01jfsb": "Thriller", "/m/06n90": "Science Fiction", "/m/03npn": "Horror", "/m/03k9fj": "Adventure", "/m/0fdjb": "Supernatural", "/m/02kdv5l": "Action", "/m/09zvmj": "Space western"}
Parsed genres: ['Thriller', 'Science Fiction', 'Horror', 'Adventure', 'Supernatural', 'Action', 'Space western']
--------------------------------------------------
Raw genre data: {"/m/02n4kr": "Mystery", "/m/03bxz7": "Biographical film", "/m/07s9rl0": "Drama", "/m/0hj3n01": "Crime Drama"}
Parsed genres: ['Mystery', 'Biographical film', 'Drama', 'Crime Drama']
--------------------------------------------------
Raw genre data: {"/m/0lsxr": "Crime Fiction", "/m/07s9rl0": "Drama"}
Parsed genres: ['Crime Fiction', 'Drama']
--------------------------------------------------
Cleaning summaries...
Extracting genres...
Entries with non-empty genre lists: 79447
Merging datasets...
Merged dataset c

genre model training

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import joblib
import json

def train_genre_prediction_model(data_path, model_output_dir):
    """
    Train a genre prediction model using movie summaries
    
    Args:
        data_path: Path to the cleaned movie dataset
        model_output_dir: Directory to save the trained model and related files
    """
    print("Loading cleaned movie data...")
    # Load the cleaned dataset
    df = pd.read_csv(data_path)
    
    # Check if GenreList is a string representation of a list
    if isinstance(df['GenreList'].iloc[0], str):
        df['GenreList'] = df['GenreList'].apply(json.loads)
    
    print(f"Dataset loaded with {len(df)} entries")
    
    # Prepare feature and target variables
    X = df['CleanedSummary']
    y = df['GenreList']
    
    # Create output directory if it doesn't exist
    os.makedirs(model_output_dir, exist_ok=True)
    
    # Multi-label binarizer
    print("Encoding genre labels...")
    mlb = MultiLabelBinarizer()
    y_encoded = mlb.fit_transform(y)
    
    # Print genre statistics
    genre_counts = y_encoded.sum(axis=0)
    genres = mlb.classes_
    genre_stats = pd.DataFrame({'Genre': genres, 'Count': genre_counts})
    genre_stats = genre_stats.sort_values('Count', ascending=False)
    
    print("\nGenre distribution (top 10):")
    print(genre_stats.head(10))
    
    # Save genre distribution figure
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Count', y='Genre', data=genre_stats.head(20))
    plt.title('Top 20 Movie Genres Distribution')
    plt.tight_layout()
    plt.savefig(os.path.join(model_output_dir, 'genre_distribution.png'))
    plt.close()
    
    # Split the data
    print("\nSplitting data into training and test sets (80/20 split)...")
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded[:, 0:2] if y_encoded.shape[1] > 2 else None
    )
    
    print(f"Training set size: {X_train.shape[0]} samples")
    print(f"Test set size: {X_test.shape[0]} samples")
    
    # Feature extraction with TF-IDF
    print("\nExtracting features using TF-IDF...")
    tfidf_vectorizer = TfidfVectorizer(max_features=5000, min_df=5, max_df=0.8)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    print(f"Feature matrix shape: {X_train_tfidf.shape}")
    
    # Build multi-label classification model
    print("\nTraining multi-label classification model...")
    classifier = OneVsRestClassifier(LinearSVC(C=1.0, class_weight='balanced'))
    classifier.fit(X_train_tfidf, y_train)
    
    # Predictions
    print("Making predictions and evaluating the model...")
    y_pred_train = classifier.predict(X_train_tfidf)
    y_pred_test = classifier.predict(X_test_tfidf)
    
    # Evaluation metrics
    print("\nTraining Set Metrics:")
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train, average='micro')
    train_recall = recall_score(y_train, y_pred_train, average='micro')
    train_f1 = f1_score(y_train, y_pred_train, average='micro')
    
    print(f"Accuracy: {train_accuracy:.4f}")
    print(f"Precision: {train_precision:.4f}")
    print(f"Recall: {train_recall:.4f}")
    print(f"F1 Score: {train_f1:.4f}")
    
    print("\nTest Set Metrics:")
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test, average='micro')
    test_recall = recall_score(y_test, y_pred_test, average='micro')
    test_f1 = f1_score(y_test, y_pred_test, average='micro')
    
    print(f"Accuracy: {test_accuracy:.4f}")
    print(f"Precision: {test_precision:.4f}")
    print(f"Recall: {test_recall:.4f}")
    print(f"F1 Score: {test_f1:.4f}")
    
    # Plot confusion matrix for top genres
    def plot_confusion_matrices(y_true, y_pred, classes, title, filename):
        """Plot confusion matrices for the top N genres"""
        n_classes = min(10, len(classes))  # Top 10 genres
        
        plt.figure(figsize=(15, 12))
        for i in range(n_classes):
            plt.subplot(3, 4, i + 1)
            cm = confusion_matrix(y_true[:, i], y_pred[:, i])
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
            plt.title(f'Genre: {classes[i]}')
            plt.xlabel('Predicted')
            plt.ylabel('True')
        
        plt.tight_layout()
        plt.suptitle(title, fontsize=16)
        plt.subplots_adjust(top=0.92)
        plt.savefig(filename)
        plt.close()
    
    # Plot confusion matrices for the top genres
    top_genres_indices = genre_counts.argsort()[::-1][:10]
    top_genres = [genres[i] for i in top_genres_indices]
    
    plot_confusion_matrices(
        y_test, y_pred_test, 
        top_genres, 
        'Confusion Matrices for Top Genres (Test Set)',
        os.path.join(model_output_dir, 'confusion_matrices_test.png')
    )
    
    # Save the metrics
    metrics = {
        'train': {
            'accuracy': train_accuracy,
            'precision': train_precision,
            'recall': train_recall,
            'f1': train_f1
        },
        'test': {
            'accuracy': test_accuracy,
            'precision': test_precision,
            'recall': test_recall,
            'f1': test_f1
        }
    }
    
    with open(os.path.join(model_output_dir, 'metrics.json'), 'w') as f:
        json.dump(metrics, f, indent=4)
    
    # Save the model and related components
    print("\nSaving model and related components...")
    joblib.dump(classifier, os.path.join(model_output_dir, 'genre_classifier.pkl'))
    joblib.dump(tfidf_vectorizer, os.path.join(model_output_dir, 'tfidf_vectorizer.pkl'))
    joblib.dump(mlb, os.path.join(model_output_dir, 'multilabel_binarizer.pkl'))
    
    print(f"Model and components saved to {model_output_dir}")
    
    return classifier, tfidf_vectorizer, mlb, metrics

def predict_genre(summary, classifier, vectorizer, mlb):
    """
    Predict the genre(s) of a movie based on its summary
    
    Args:
        summary: Movie summary text
        classifier: Trained genre classifier
        vectorizer: TF-IDF vectorizer
        mlb: Multi-label binarizer
        
    Returns:
        List of predicted genres
    """
    # Vectorize the summary
    summary_vector = vectorizer.transform([summary])
    
    # Predict genres
    genre_prediction = classifier.predict(summary_vector)
    
    # Decode prediction
    predicted_genres = mlb.inverse_transform(genre_prediction)[0]
    
    return list(predicted_genres)

def main():
    """Main function to train the genre prediction model"""

    # File paths - For Jupyter/IPython environment
    # Use current working directory instead of __file__
    BASE_DIR = os.getcwd()
    data_path = os.path.join(BASE_DIR, 'output', 'cleaned_movie_data.csv')
    model_output_dir = os.path.join(BASE_DIR, 'model')
    
    # Alternative: Hardcode paths if needed
    # data_path = 'output/cleaned_movie_data.csv'
    # model_output_dir = 'model'
    
    print(f"Looking for data at: {data_path}")
    
    # Check if the cleaned data file exists
    if not os.path.exists(data_path):
        print(f"Error: {data_path} not found. Please run the data preprocessing script first.")
        print("Current working directory:", BASE_DIR)
        print("Available files in current directory:", os.listdir(BASE_DIR))
        if os.path.exists('output'):
            print("Files in output directory:", os.listdir('output'))
        return
    
    # Train the model
    classifier, vectorizer, mlb, metrics = train_genre_prediction_model(data_path, model_output_dir)
    
    # Test the model with a sample summary
    sample_summary = "A young hero embarks on an epic journey to save the world from evil forces that threaten to destroy everything he loves."
    predicted_genres = predict_genre(sample_summary, classifier, vectorizer, mlb)
    
    print("\nTesting the model with a sample summary:")
    print(f"Summary: {sample_summary}")
    print(f"Predicted genres: {predicted_genres}")
    
    print("\nModel training and evaluation complete!")

if __name__ == "__main__":
    main()

Looking for data at: c:\Users\Dell\Documents\SEM6\ai\proj\extra\output\cleaned_movie_data.csv
Loading cleaned movie data...
Dataset loaded with 41793 entries
Encoding genre labels...

Genre distribution (top 10):
               Genre  Count
114            Drama  19134
75            Comedy  10467
288     Romance Film   6666
343         Thriller   6530
2             Action   5868
359     World cinema   5153
98     Crime Fiction   4275
189           Horror   4082
43   Black-and-white   3731
196            Indie   3668

Splitting data into training and test sets (80/20 split)...
Training set size: 33434 samples
Test set size: 8359 samples

Extracting features using TF-IDF...
Feature matrix shape: (33434, 5000)

Training multi-label classification model...


C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\multiclass.py:90: UserWarning: Label not 48 is present in all training examples.
  warnings.warn(
C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\multiclass.py:90: UserWarning: Label not 61 is present in all training examples.
  warnings.warn(
C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\multiclass.py:90: UserWarning: Label not 66 is present in all training examples.
  warnings.warn(
C:\Users\Dell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\multiclass.py:90: UserWarning: Label not 74 is present in all training examples.
  warnings.warn(


Making predictions and evaluating the model...

Training Set Metrics:
Accuracy: 0.0637
Precision: 0.5453
Recall: 0.9469
F1 Score: 0.6920

Test Set Metrics:
Accuracy: 0.0049
Precision: 0.3046
Recall: 0.5151
F1 Score: 0.3829

Saving model and related components...
Model and components saved to c:\Users\Dell\Documents\SEM6\ai\proj\extra\model

Testing the model with a sample summary:
Summary: A young hero embarks on an epic journey to save the world from evil forces that threaten to destroy everything he loves.
Predicted genres: ['Adventure', 'Short Film']

Model training and evaluation complete!


translation and audio

In [5]:
import pandas as pd
import os
import time
import tqdm
from googletrans import Translator
from gtts import gTTS
import json

# Language settings
LANGUAGE_MAP = {
    'arabic': {'code': 'ar', 'tts_code': 'ar'},
    'urdu': {'code': 'ur', 'tts_code': 'ur'},
    'korean': {'code': 'ko', 'tts_code': 'ko'},
    'english': {'code': 'en', 'tts_code': 'en'},
    'spanish': {'code': 'es', 'tts_code': 'es'},
    'french': {'code': 'fr', 'tts_code': 'fr'}
}

def translate_text(text, target_language, translator=None):
    """
    Translate text to target language
    
    Args:
        text: Text to translate
        target_language: Target language code
        translator: Translator instance (optional, will create new one if None)
        
    Returns:
        Translated text or None if translation failed
    """
    if translator is None:
        translator = Translator()
    
    try:
        translation = translator.translate(text, dest=target_language)
        return translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        return None

def text_to_speech(text, language, filename):
    """
    Convert text to speech and save as audio file
    
    Args:
        text: Text to convert to speech
        language: Language code for TTS
        filename: Output audio filename
        
    Returns:
        True if successful, False otherwise
    """
    try:
        tts = gTTS(text=text, lang=language, slow=False)
        tts.save(filename)
        return True
    except Exception as e:
        print(f"TTS error: {e}")
        return False

def process_movies_for_translation(df, num_movies=50, max_summary_length=500):
    """
    Process movies for translation and audio conversion
    
    Args:
        df: DataFrame containing movie data
        num_movies: Number of movies to process
        max_summary_length: Maximum length of summary to process
        
    Returns:
        List of processed movie IDs
    """
    # Create directories for storing translations and audio
    os.makedirs('translations', exist_ok=True)
    os.makedirs('audio_files', exist_ok=True)
    
    # Initialize translator
    translator = Translator()
    
    # Process subset of movies
    processed_movies = df.head(num_movies)
    processed_ids = []
    
    # Process each movie
    for idx, row in tqdm.tqdm(processed_movies.iterrows(), total=len(processed_movies), desc="Processing movies"):
        movie_id = str(row['WikipediaID'])
        summary = row['CleanedSummary']
        
        # Limit summary length for API constraints
        if len(summary) > max_summary_length:
            summary = summary[:max_summary_length]
        
        title = row['Title']
        
        print(f"\nProcessing movie: {title}")
        
        all_translations_successful = True
        
        # Process each language
        for lang_name, lang_info in LANGUAGE_MAP.items():
            # Skip if files already exist
            translation_file = f"translations/{movie_id}_{lang_name}.txt"
            audio_file = f"audio_files/{movie_id}_{lang_name}.mp3"
            
            if os.path.exists(translation_file) and os.path.exists(audio_file):
                print(f"  {lang_name} translation and audio already exist for {title}")
                continue
            
            # For English, skip translation and use original summary
            if lang_name == 'english':
                # Save original text
                with open(translation_file, "w", encoding="utf-8") as f:
                    f.write(summary)
                
                # Convert to audio
                print(f"  Converting to {lang_name} audio (no translation needed)...")
                success = text_to_speech(summary, lang_info['tts_code'], audio_file)
                
                if success:
                    print(f"  Created audio file for {title} in {lang_name}")
                else:
                    print(f"  Failed to create audio for {title} in {lang_name}")
                    all_translations_successful = False
            else:
                # Translate text for non-English languages
                print(f"  Translating to {lang_name}...")
                translated_text = translate_text(summary, lang_info['code'], translator)
                
                if translated_text:
                    # Save translation
                    with open(translation_file, "w", encoding="utf-8") as f:
                        f.write(translated_text)
                    
                    # Convert to audio
                    print(f"  Converting to {lang_name} audio...")
                    success = text_to_speech(translated_text, lang_info['tts_code'], audio_file)
                    
                    if success:
                        print(f"  Created audio file for {title} in {lang_name}")
                    else:
                        print(f"  Failed to create audio for {title} in {lang_name}")
                        all_translations_successful = False
                else:
                    print(f"  Translation to {lang_name} failed for {title}")
                    all_translations_successful = False
            
            # Sleep to avoid API rate limits
            time.sleep(1)
        
        if all_translations_successful:
            processed_ids.append(movie_id)
        
    return processed_ids

# File path configuration
# Change this to the location of your cleaned movie data CSV file
data_path = 'output/cleaned_movie_data.csv'

# Check if the data file exists
if not os.path.exists(data_path):
    print(f"Error: {data_path} not found")
else:
    # Load data
    print(f"Loading movie data from {data_path}...")
    df = pd.read_csv(data_path)
    
    # Set parameters
    num_movies = 50  # Change this number if you want to process a different number of movies
    max_summary_length = 500  # Maximum length of summary to process
    
    print(f"Processing {num_movies} movies for translation and audio conversion...")
    processed_ids = process_movies_for_translation(df, num_movies, max_summary_length)
    
    # Save list of processed IDs
    with open('processed_movie_ids.json', 'w') as f:
        json.dump(processed_ids, f)
    
    print(f"\nProcessed {len(processed_ids)} movies successfully")
    print(f"Processed movie IDs saved to processed_movie_ids.json")

Loading movie data from output/cleaned_movie_data.csv...
Processing 50 movies for translation and audio conversion...


Processing movies:   0%|          | 0/50 [00:00<?, ?it/s]


Processing movie: Taxi Blues
  Translating to arabic...
  Converting to arabic audio...
  Created audio file for Taxi Blues in arabic
  Translating to urdu...
  Converting to urdu audio...
  Created audio file for Taxi Blues in urdu
  Translating to korean...
  Converting to korean audio...
  Created audio file for Taxi Blues in korean
  Converting to english audio (no translation needed)...
  Created audio file for Taxi Blues in english
  Translating to spanish...
  Converting to spanish audio...
  Created audio file for Taxi Blues in spanish
  Translating to french...
  Converting to french audio...
  Created audio file for Taxi Blues in french


Processing movies:   2%|▏         | 1/50 [00:20<16:50, 20.62s/it]


Processing movie: The Hunger Games
  Translating to arabic...
  Converting to arabic audio...
  Created audio file for The Hunger Games in arabic
  Translating to urdu...
  Converting to urdu audio...
  Created audio file for The Hunger Games in urdu
  Translating to korean...
  Converting to korean audio...
  Created audio file for The Hunger Games in korean
  Converting to english audio (no translation needed)...
  Created audio file for The Hunger Games in english
  Translating to spanish...
  Converting to spanish audio...
  Created audio file for The Hunger Games in spanish
  Translating to french...
  Converting to french audio...
  Created audio file for The Hunger Games in french


...


Processing movie: The Storm Riders
  Translating to arabic...
  Converting to arabic audio...
  Created audio file for The Storm Riders in arabic
  Translating to urdu...
  Converting to urdu audio...
  Created audio file for The Storm Riders in urdu
  Translating to korean...
  Converting to korean audio...
  Created audio file for The Storm Riders in korean
  Converting to english audio (no translation needed)...
  Created audio file for The Storm Riders in english
  Translating to spanish...
  Converting to spanish audio...
  Created audio file for The Storm Riders in spanish
  Translating to french...
  Converting to french audio...
  Created audio file for The Storm Riders in french


Processing movies:  98%|█████████▊| 49/50 [27:43<00:33, 33.02s/it]


Processing movie: Against Her Will: An Incident in Baltimore
  Translating to arabic...
  Converting to arabic audio...
  Created audio file for Against Her Will: An Incident in Baltimore in arabic
  Translating to urdu...
  Converting to urdu audio...
  Created audio file for Against Her Will: An Incident in Baltimore in urdu
  Translating to korean...
  Converting to korean audio...
  Created audio file for Against Her Will: An Incident in Baltimore in korean
  Converting to english audio (no translation needed)...
  Created audio file for Against Her Will: An Incident in Baltimore in english
  Translating to spanish...
  Converting to spanish audio...
  Created audio file for Against Her Will: An Incident in Baltimore in spanish
  Translating to french...
  Converting to french audio...
  Created audio file for Against Her Will: An Incident in Baltimore in french


Processing movies: 100%|██████████| 50/50 [28:04<00:00, 33.68s/it]


Processed 46 movies successfully
Processed movie IDs saved to processed_movie_ids.json


gui

In [9]:
# Cell 1: Import necessary libraries
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext, filedialog
from PIL import Image, ImageTk
import pandas as pd
import os
import pygame
from googletrans import Translator
from gtts import gTTS
import joblib
import threading
import time

# Cell 2: FilmceptionGUI class definition
class FilmceptionGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Filmception - AI Movie Summary System")
        self.root.geometry("900x700")
        self.root.resizable(True, True)
        
        # Initialize system components
        self.translator = Translator()
        self.language_map = {
            '1': {'name': 'Arabic', 'code': 'ar', 'tts_code': 'ar'},
            '2': {'name': 'Urdu', 'code': 'ur', 'tts_code': 'ur'},
            '3': {'name': 'Korean', 'code': 'ko', 'tts_code': 'ko'},
            '4': {'name': 'English', 'code': 'en', 'tts_code': 'en'},
            '5': {'name': 'Spanish', 'code': 'es', 'tts_code': 'es'},
            '6': {'name': 'French', 'code': 'fr', 'tts_code': 'fr'},
        }
        
        # Audio control variables
        self.current_audio_file = None
        self.audio_playing = False
        self.audio_paused = False
        
        # Create StringVar for status before using it
        self.status_var = tk.StringVar(self.root)
        self.status_var.set("Ready")
        
        # Create StringVar for now playing before using it
        self.now_playing_var = tk.StringVar(self.root)
        self.now_playing_var.set("No audio loaded")
        
        # Load data and models
        self.load_resources()
        
        # Initialize pygame for audio
        pygame.mixer.init()
        
        # Create GUI components
        self.create_widgets()
        
        # Create directories if they don't exist
        os.makedirs('translations', exist_ok=True)
        os.makedirs('audio_files', exist_ok=True)

# Cell 3: Add load_resources method
    def load_resources(self):
        """Load data and models"""
        base_dir = os.getcwd()
        self.data_path = os.path.join(base_dir, 'output', 'cleaned_movie_data.csv')
        self.model_dir = os.path.join(base_dir, 'model')

        try:
            self.movies_df = pd.read_csv(self.data_path)
            print(f"Movie data loaded from {self.data_path}")
        except FileNotFoundError:
            self.movies_df = None
            print(f"Error: Could not find dataset at {self.data_path}")
        
        try:
            self.classifier = joblib.load(os.path.join(self.model_dir, 'genre_classifier.pkl'))
            self.vectorizer = joblib.load(os.path.join(self.model_dir, 'tfidf_vectorizer.pkl'))
            self.mlb = joblib.load(os.path.join(self.model_dir, 'multilabel_binarizer.pkl'))
            print("Models loaded from", self.model_dir)
        except FileNotFoundError as e:
            self.classifier = None
            self.vectorizer = None
            self.mlb = None
            print(f"Error loading models: {e}")

# Cell 4: Add create_widgets method
    def create_widgets(self):
        """Create all GUI widgets"""
        # Create main container
        self.main_frame = ttk.Frame(self.root, padding="10")
        self.main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Header
        header_frame = ttk.Frame(self.main_frame)
        header_frame.pack(fill=tk.X, pady=10)
        
        # Logo and title
        try:
            img = Image.open("logo.png").resize((80, 80))
            self.logo = ImageTk.PhotoImage(img)
            logo_label = ttk.Label(header_frame, image=self.logo)
            logo_label.pack(side=tk.LEFT, padx=10)
        except:
            pass
        
        title_label = ttk.Label(header_frame, text="FILMCEPTION", font=('Helvetica', 18, 'bold'))
        title_label.pack(side=tk.LEFT, padx=10)
        subtitle_label = ttk.Label(header_frame, text="AI Movie Summary Translator & Genre Classifier", font=('Helvetica', 12))
        subtitle_label.pack(side=tk.LEFT)
        
        # Navigation tabs
        self.notebook = ttk.Notebook(self.main_frame)
        self.notebook.pack(fill=tk.BOTH, expand=True, pady=10)
        
        # Create tabs
        self.create_new_summary_tab()
        self.create_preprocessed_tab()
        self.create_about_tab()
        
        # Status bar
        self.status_bar = ttk.Label(self.main_frame, textvariable=self.status_var, relief=tk.SUNKEN)
        self.status_bar.pack(fill=tk.X, pady=(5, 0))
        
        # Audio control panel (initially hidden)
        self.create_audio_controls()

# Cell 5: Add create_audio_controls method
    def create_audio_controls(self):
        """Create audio control panel"""
        self.audio_frame = ttk.LabelFrame(self.main_frame, text="Audio Controls")
        # Note: We don't pack it yet - it will be packed when needed
        
        # Audio controls
        controls_frame = ttk.Frame(self.audio_frame)
        controls_frame.pack(pady=5)
        
        # Play button
        self.play_button = ttk.Button(controls_frame, text="▶ Play", width=8, command=self.toggle_play_pause)
        self.play_button.pack(side=tk.LEFT, padx=5)
        
        # Stop button
        self.stop_button = ttk.Button(controls_frame, text="■ Stop", width=8, command=self.stop_audio)
        self.stop_button.pack(side=tk.LEFT, padx=5)
        
        # Now playing label
        ttk.Label(self.audio_frame, textvariable=self.now_playing_var, font=('Helvetica', 10, 'italic')).pack(pady=5)

# Cell 6: Add audio control methods
    def toggle_play_pause(self):
        """Toggle between play and pause states"""
        if not self.current_audio_file:
            return
            
        if not self.audio_playing:
            # Start playing
            try:
                if self.audio_paused:
                    pygame.mixer.music.unpause()
                else:
                    pygame.mixer.music.load(self.current_audio_file)
                    pygame.mixer.music.play()
                    self.start_audio_monitor()
                
                self.audio_playing = True
                self.audio_paused = False
                self.play_button.config(text="❚❚ Pause")
                self.status_var.set("Playing audio...")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to play audio: {str(e)}")
                
        else:
            # Pause playing
            pygame.mixer.music.pause()
            self.audio_playing = False
            self.audio_paused = True
            self.play_button.config(text="▶ Play")
            self.status_var.set("Audio paused")
    
    def stop_audio(self):
        """Stop audio playback"""
        if self.current_audio_file:
            pygame.mixer.music.stop()
            self.audio_playing = False
            self.audio_paused = False
            self.play_button.config(text="▶ Play")
            self.status_var.set("Audio stopped")
    
    def start_audio_monitor(self):
        """Start a thread to monitor audio playback status"""
        def monitor():
            while self.audio_playing:
                if not pygame.mixer.music.get_busy() and not self.audio_paused:
                    # Audio finished playing
                    self.audio_playing = False
                    # Use after() to safely update the UI from a different thread
                    self.root.after(100, lambda: self.play_button.config(text="▶ Play"))
                    self.root.after(100, lambda: self.status_var.set("Audio playback complete"))
                    break
                time.sleep(0.1)
                
        monitor_thread = threading.Thread(target=monitor)
        monitor_thread.daemon = True
        monitor_thread.start()

# Cell 7: Add tab creation methods
    def create_new_summary_tab(self):
        """Tab for entering new summaries"""
        tab = ttk.Frame(self.notebook)
        self.notebook.add(tab, text="New Summary")
        
        # Title input
        ttk.Label(tab, text="Movie Title:").grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)
        self.title_entry = ttk.Entry(tab, width=50)
        self.title_entry.grid(row=0, column=1, columnspan=2, sticky=tk.W, padx=5, pady=5)
        
        # Summary input
        ttk.Label(tab, text="Movie Summary:").grid(row=1, column=0, sticky=tk.NW, padx=5, pady=5)
        self.summary_text = scrolledtext.ScrolledText(tab, width=70, height=15, wrap=tk.WORD)
        self.summary_text.grid(row=1, column=1, columnspan=2, padx=5, pady=5)
        
        # Buttons
        button_frame = ttk.Frame(tab)
        button_frame.grid(row=2, column=1, columnspan=2, sticky=tk.E, pady=10)
        
        ttk.Button(button_frame, text="Clear", command=self.clear_inputs).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Convert to Audio", command=self.show_audio_options).pack(side=tk.LEFT, padx=5)
        ttk.Button(button_frame, text="Predict Genre", command=self.predict_genre_gui).pack(side=tk.LEFT, padx=5)
    
    def create_preprocessed_tab(self):
        """Tab for selecting preprocessed movies"""
        tab = ttk.Frame(self.notebook)
        self.notebook.add(tab, text="Preprocessed Movies")
        
        if self.movies_df is None:
            ttk.Label(tab, text="No preprocessed movies available. Please load the dataset.").pack(pady=50)
            return
        
        # Filter movies that have been processed
        self.available_movies = self.movies_df.head(50)  # Limit to 50 for demo
        
        # Movie selection
        ttk.Label(tab, text="Select a movie:").pack(pady=5)
        
        self.movie_listbox = tk.Listbox(tab, height=15, width=80)
        self.movie_listbox.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)
        
        # Populate listbox
        for _, row in self.available_movies.iterrows():
            self.movie_listbox.insert(tk.END, row['Title'])
        
        # Buttons
        button_frame = ttk.Frame(tab)
        button_frame.pack(pady=10)
        
        ttk.Button(button_frame, text="Select Movie", command=self.select_preprocessed_movie).pack(side=tk.LEFT, padx=5)
    
    def create_about_tab(self):
        """About tab with project information"""
        tab = ttk.Frame(self.notebook)
        self.notebook.add(tab, text="About")
        
        about_text = """
        Filmception - AI Movie Summary System
        
        This application provides:
        - Translation of movie summaries to multiple languages
        - Audio conversion of translated summaries
        - Genre prediction based on movie summaries
        
        Languages Supported:
        - Arabic
        - Urdu
        - Korean
        - English
        - Spanish
        - French
        
        Developed for the Artificial Intelligence course
        Department of Cyber Security
        Spring 2025
        """
        
        ttk.Label(tab, text=about_text, justify=tk.LEFT).pack(pady=20, padx=20)
        
        # Add credits
        ttk.Label(tab, text="Developed by: Fatima Babar and Osaed Jafar", font=('Helvetica', 10)).pack(side=tk.BOTTOM, pady=10)

# Cell 8: Add input handling methods
    def clear_inputs(self):
        """Clear all input fields"""
        self.title_entry.delete(0, tk.END)
        self.summary_text.delete(1.0, tk.END)
        self.status_var.set("Inputs cleared")
    
    def show_audio_options(self):
        """Show language options for audio conversion"""
        title = self.title_entry.get()
        summary = self.summary_text.get(1.0, tk.END).strip()
        
        if not summary:
            messagebox.showerror("Error", "Please enter a movie summary first")
            return
        
        # Create popup window
        popup = tk.Toplevel(self.root)
        popup.title("Language Selection")
        popup.geometry("400x450")
        popup.resizable(True, False)
        
        ttk.Label(popup, text="Select Language for Audio:", font=('Helvetica', 12)).pack(pady=10)
        
        # Language buttons with more padding and in a scrollable frame if needed
        languages_frame = ttk.Frame(popup)
        languages_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        # Language buttons
        for key, lang in self.language_map.items():
            ttk.Button(languages_frame, text=lang['name'], 
                    command=lambda l=lang: self.convert_to_audio(summary, l, title, popup)).pack(fill=tk.X, pady=8)
        
        ttk.Button(popup, text="Cancel", command=popup.destroy).pack(fill=tk.X, padx=20, pady=15)

# Cell 9: Add audio conversion and translation methods
    def convert_to_audio(self, text, lang_info, title, popup):
        """Convert text to audio in selected language"""
        popup.destroy()
        self.status_var.set(f"Processing summary for {lang_info['name']}...")
        self.root.update()
        
        try:
            # Use original text if English is selected, otherwise translate
            if lang_info['code'] == 'en':
                self.status_var.set("Using original English text...")
                translated_text = text
            else:
                self.status_var.set(f"Translating text to {lang_info['name']}...")
                translated_text = self.translator.translate(text, dest=lang_info['code']).text
            
            # Use a safe filename by replacing spaces with underscores
            safe_title = title.replace(' ', '_') if title else 'untitled'
            
            # Generate audio file
            audio_file = f"audio_files/{safe_title}_{lang_info['name'].lower()}.mp3"
            self.status_var.set(f"Generating audio for {title} in {lang_info['name']}...")
            
            # Ensure audio_files directory exists
            os.makedirs('audio_files', exist_ok=True)
            
            # Create TTS with explicit error handling
            try:
                tts = gTTS(text=translated_text, lang=lang_info['tts_code'], slow=False)
                tts.save(audio_file)
                self.status_var.set(f"Audio generated for {title} in {lang_info['name']}")
                
                # Update audio control panel
                self.now_playing_var.set(f"Now Playing: {title} ({lang_info['name']})")
                self.current_audio_file = audio_file
                
                # Show the audio frame - FIXED PART
                if not self.audio_frame.winfo_ismapped():
                    self.audio_frame.pack(fill=tk.X, pady=10, before=self.status_bar)
                
                # Start playing
                self.toggle_play_pause()
                
                # Show translation
                if lang_info['code'] != 'en' and messagebox.askyesno("Translation", "Would you like to see the text translation?"):
                    self.show_translation(translated_text, lang_info['name'])
                    
            except Exception as tts_error:
                messagebox.showerror("TTS Error", f"Failed to generate audio: {str(tts_error)}")
                self.status_var.set(f"TTS failed for {lang_info['name']}")
                
        except Exception as e:
            messagebox.showerror("Error", f"Failed to process {lang_info['name']}: {str(e)}")
            self.status_var.set(f"{lang_info['name']} processing failed")
    
    def play_audio(self, audio_file):
        """Set up and play audio file"""
        # Update audio control panel
        self.now_playing_var.set(f"Now Playing: {os.path.basename(audio_file)}")
        self.current_audio_file = audio_file
        
        # Make sure audio frame is displayed - FIXED PART
        if not self.audio_frame.winfo_ismapped():
            self.audio_frame.pack(fill=tk.X, pady=10, before=self.status_bar)
        
        # Start playing
        self.toggle_play_pause()
    
    def show_translation(self, text, language):
        """Show translation in a new window"""
        popup = tk.Toplevel(self.root)
        popup.title(f"{language} Translation")
        popup.geometry("600x400")
        
        ttk.Label(popup, text=f"{language} Translation:", font=('Helvetica', 12, 'bold')).pack(pady=10)
        
        text_box = scrolledtext.ScrolledText(popup, width=70, height=20, wrap=tk.WORD)
        text_box.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)
        text_box.insert(tk.END, text)
        text_box.config(state=tk.DISABLED)
        
        ttk.Button(popup, text="Close", command=popup.destroy).pack(pady=10)

# Cell 10: Add genre prediction methods
    def predict_genre_gui(self):
        """Predict genre for the entered summary"""
        summary = self.summary_text.get(1.0, tk.END).strip()
        title = self.title_entry.get()
        
        if not summary:
            messagebox.showerror("Error", "Please enter a movie summary first")
            return
        
        if self.classifier is None or self.vectorizer is None or self.mlb is None:
            messagebox.showerror("Error", "Genre prediction model not available")
            return
        
        self.status_var.set("Predicting genre...")
        self.root.update()
        
        try:
            # Vectorize the summary
            summary_vector = self.vectorizer.transform([summary])
            
            # Predict genres
            genre_prediction = self.classifier.predict(summary_vector)
            
            # Decode prediction
            predicted_genres = self.mlb.inverse_transform(genre_prediction)[0]
            
            # Show results
            self.show_genre_results(title, predicted_genres)
            
            self.status_var.set("Genre prediction complete")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to predict genre: {str(e)}")
            self.status_var.set("Genre prediction failed")
    
    def show_genre_results(self, title, genres):
        """Show genre prediction results"""
        popup = tk.Toplevel(self.root)
        popup.title("Genre Prediction Results")
        popup.geometry("400x300")
        
        if title:
            ttk.Label(popup, text=f"Title: {title}", font=('Helvetica', 12, 'bold')).pack(pady=10)
        
        if genres:
            ttk.Label(popup, text="Predicted Genres:", font=('Helvetica', 11, 'bold')).pack()
            
            for i, genre in enumerate(genres, 1):
                ttk.Label(popup, text=f"{i}. {genre}").pack()
        else:
            ttk.Label(popup, text="No specific genre could be predicted.").pack()
        
        ttk.Button(popup, text="Close", command=popup.destroy).pack(pady=20)

# Cell 11: Add preprocessed movie handling methods
    def select_preprocessed_movie(self):
        """Handle selection of preprocessed movie"""
        selection = self.movie_listbox.curselection()
        if not selection:
            messagebox.showerror("Error", "Please select a movie first")
            return
        
        index = selection[0]
        movie = self.available_movies.iloc[index]
        
        # Show movie options
        self.show_movie_options(movie)
    
    def show_movie_options(self, movie):
        """Show options for selected preprocessed movie"""
        popup = tk.Toplevel(self.root)
        popup.title(f"Options for {movie['Title']}")
        popup.geometry("400x300")
        
        ttk.Label(popup, text=f"Movie: {movie['Title']}", font=('Helvetica', 12, 'bold')).pack(pady=10)
        
        # Buttons
        ttk.Button(popup, text="Convert Summary to Audio", 
                  command=lambda: self.show_audio_options_preprocessed(movie, popup)).pack(fill=tk.X, padx=50, pady=10)
        
        ttk.Button(popup, text="Predict Genre", 
                  command=lambda: self.predict_genre_preprocessed(movie['CleanedSummary'], movie['Title'], popup)).pack(fill=tk.X, padx=50, pady=10)
        
        ttk.Button(popup, text="Close", command=popup.destroy).pack(fill=tk.X, padx=50, pady=10)
    
    def show_audio_options_preprocessed(self, movie, parent):
        """Show audio options for preprocessed movie"""
        parent.destroy()
        
        popup = tk.Toplevel(self.root)
        popup.title(f"Audio Options for {movie['Title']}")
        popup.geometry("400x450")
        popup.resizable(True, False)

        ttk.Label(popup, text="Select Language:",font=('Helvetica', 12)).pack(pady=10)
        
       # Language buttons in a frame
        languages_frame = ttk.Frame(popup)
        languages_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=10)
        
        for key, lang in self.language_map.items():
            ttk.Button(languages_frame, text=lang['name'], 
                    command=lambda l=lang: self.handle_preprocessed_audio(movie, l, popup)).pack(fill=tk.X, pady=8)
        
        ttk.Button(popup, text="Cancel", command=popup.destroy).pack(fill=tk.X, padx=20, pady=15)
    
    def handle_preprocessed_audio(self, movie, lang_info, popup):
        """Handle audio for preprocessed movie"""
        popup.destroy()
        movie_id = str(movie['WikipediaID']) if 'WikipediaID' in movie else movie['Title'].replace(' ', '_')
        summary = movie['CleanedSummary']
        title = movie['Title']
        
        audio_file = f"audio_files/{movie_id}_{lang_info['name'].lower()}.mp3"
        translation_file = f"translations/{movie_id}_{lang_info['name'].lower()}.txt"
        
        # Check if files exist
        if os.path.exists(audio_file) and os.path.exists(translation_file):
            self.status_var.set(f"Playing existing {lang_info['name']} audio...")
            self.root.update()
            
            # Play audio with controls
            self.play_audio(audio_file)
            
            if lang_info['code'] != 'en' and messagebox.askyesno("Translation", "Would you like to see the text translation?"):
                with open(translation_file, 'r', encoding='utf-8') as f:
                    self.show_translation(f.read(), lang_info['name'])
        else:
            self.status_var.set(f"Processing summary for {lang_info['name']}...")
            self.root.update()
            
            try:
                # Use original text if English is selected, otherwise translate
                if lang_info['code'] == 'en':
                    self.status_var.set("Using original English text...")
                    translated_text = summary
                else:
                    self.status_var.set(f"Translating to {lang_info['name']}...")
                    translated_text = self.translator.translate(summary, dest=lang_info['code']).text
                
                # Ensure directories exist
                os.makedirs('translations', exist_ok=True)
                os.makedirs('audio_files', exist_ok=True)
                
                # Save translation
                with open(translation_file, "w", encoding="utf-8") as f:
                    f.write(translated_text)
                
                # Generate audio
                tts = gTTS(text=translated_text, lang=lang_info['tts_code'], slow=False)
                tts.save(audio_file)
                
                self.status_var.set(f"Audio generated for {title} in {lang_info['name']}")
                self.root.update()
                
                # Play audio with controls
                self.now_playing_var.set(f"Now Playing: {title} ({lang_info['name']})")
                self.current_audio_file = audio_file
                if not self.audio_frame.winfo_ismapped():
                    self.audio_frame.pack(fill=tk.X, pady=10, before=self.status_bar)
                self.toggle_play_pause()
                
                # Show translation
                if lang_info['code'] != 'en' and messagebox.askyesno("Translation", "Would you like to see the text translation?"):
                    self.show_translation(translated_text, lang_info['name'])
                    
            except Exception as e:
                messagebox.showerror("Error", f"Failed to process audio: {str(e)}")
                self.status_var.set("Audio processing failed")
    
    def predict_genre_preprocessed(self, summary, title, parent):
        """Predict genre for preprocessed movie"""
        parent.destroy()
        
        if self.classifier is None or self.vectorizer is None or self.mlb is None:
            messagebox.showerror("Error", "Genre prediction model not available")
            return
        
        self.status_var.set("Predicting genre...")
        self.root.update()
        
        try:
            # Vectorize the summary
            summary_vector = self.vectorizer.transform([summary])
            
            # Predict genres
            genre_prediction = self.classifier.predict(summary_vector)
            
            # Decode prediction
            predicted_genres = self.mlb.inverse_transform(genre_prediction)[0]
            
            # Show results
            self.show_genre_results(title, predicted_genres)
            
            self.status_var.set("Genre prediction complete")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to predict genre: {str(e)}")
            self.status_var.set("Genre prediction failed")

# Cell 12: Run the application
# This cell starts the GUI when executed
root = tk.Tk()
app = FilmceptionGUI(root)
root.mainloop()

Movie data loaded from c:\Users\Dell\Documents\SEM6\ai\proj\extra\output\cleaned_movie_data.csv
Models loaded from c:\Users\Dell\Documents\SEM6\ai\proj\extra\model
